## Tuning the parameters of our model separately for each dataset in the ETH/UCY datasets

In [1]:
import sys
import os
import dill
import json
import argparse
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv

In [29]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

def process_data(input_data, limit=None):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (7 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    if limit:
        data = data.iloc[:limit]
        data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

In [30]:
def evaluate_our_method(data, params, dataset_title='', single_output = False):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, _ = generative_model.predict(x_data, y_data, params, trajectory_length=12)

                    best_fde = None
                    best_ade = None
                    
                    if single_output:
                        avg_x = np.mean(all_pred_x, axis=0)
                        avg_y = np.mean(all_pred_y, axis=0)
                        best_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                        best_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    else:
                        for i in range(len(all_pred_x)):
                            current_pred_x = all_pred_x[i]
                            current_pred_y = all_pred_y[i]

                            fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_fde == None or fde < best_fde:
                                best_fde = fde

                            ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_ade == None or ade < best_ade:
                                best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

In [31]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results_paper_version/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [36]:
def evaluate_on_dataset(our_method_params, file, trajectron_resultset_name, evaluate_most_likely=False, limit=None):
    base_path = './raw_data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for param_idx, params in enumerate(our_method_params):
        data = pd.read_csv(base_path + file, sep='\t', index_col=False, header=None)
        data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

        data = process_data(data, limit=limit)

        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_name)
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_name, suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        num_predictable_trajectories = get_total_predictable_slices(data)
        #assert len(trajectron_fde) == num_predictable_trajectories
        #assert len(trajectron_ade) == num_predictable_trajectories

        ## Ours
        our_fde_best_of_20, our_ade_best_of_20 = evaluate_our_method(data, params, dataset_title=trajectron_resultset_name)
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
        
        if evaluate_most_likely:
            our_fde_single, our_ade_single = evaluate_our_method(data, params, dataset_title=trajectron_resultset_name, single_output=True)
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_single))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_single))

    return [
        ours_results,
        trajectron_results,
    ]

In [16]:
our_method_params = [params_eth, params_eth, params_rest, params_rest]

files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel', 
    'zara1_vel', 
    'zara2_vel'
]

NameError: name 'params_eth' is not defined

## Zara 1

In [173]:
our_method_params = []

for angle_change_prob in [0.25]:
    for angle_change_noise in [2.5]:
                
            one_param_set = {
                'NOISE': 0.05, 
                'NO_OF_TRAJECTORIES': 300, 
                'CONST_VEL_MODEL_PROB': 0.5, 
                'STOP_PROB': 0.05, 
                'DISCOUNT_AVG_PROB': 1.0, 
                'DISCOUNT_LOWER_BOUND': 0.2, 
                'VELOCITY_CHANGE_PROB': 0.2,
                'VELOCITY_CHANGE_NOISE': 0.1, 
                'ANGLE_CHANGE_PROB': angle_change_prob, 
                'ANGLE_CHANGE_NOISE': angle_change_noise, 
                'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
                'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
            }
                
                
            our_method_params.append(one_param_set)

In [174]:
res = evaluate_on_dataset(our_method_params, 'zara1/test/crowds_zara01.txt', 'zara1_vel', limit=500, evaluate_most_likely=False)

Ours - zara1_vel: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:13<00:00,  4.77it/s]


In [171]:
ours_res = res[0]
trajectron_res = res[1]

In [172]:
ours_bo20_fde = ours_res['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params[index_min])

0
Lowest FDE:  0.36381538799136276
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 200, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.2, 'VELOCITY_CHANGE_NOISE': 0.1, 'ANGLE_CHANGE_PROB': 0.25, 'ANGLE_CHANGE_NOISE': 2.5, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


In [168]:
ours_bo20_fde = ours_res['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params[index_min])

0
Lowest FDE:  0.37255842850463805
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 10000, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.2, 'VELOCITY_CHANGE_NOISE': 0.1, 'ANGLE_CHANGE_PROB': 0.25, 'ANGLE_CHANGE_NOISE': 2.5, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


In [60]:
our_params = our_method_params[index_min]
res = evaluate_on_dataset([our_params], 'zara1/test/crowds_zara01.txt', 'zara1_vel', evaluate_most_likely=False)

Ours - zara1_vel: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 864/864 [06:33<00:00,  2.20it/s]


In [61]:
print(res)

[{'BEST_OF_20': {'FDE': [0.47540723950032054], 'ADE': [0.2589786249261643]}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}, {'BEST_OF_20': {'FDE': [0.3263688810977737], 'ADE': [0.1512007682502178]}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}]


## Zara2

In [156]:
our_method_params_z2 = []

for angle_change_prob in [0.25]: # done
    for angle_change_noise in [2]: # done
        for velocity_change_prob in [0.1]: # done
            for velocity_change_noise in [0.05]: # done
                for discount_lower_bound in [0.2]: # done
                    for no_of_trajectories in [200, 500, 1000]:
                        one_param_set = {
                            'NOISE': 0.05, 
                            'NO_OF_TRAJECTORIES': no_of_trajectories, 
                            'CONST_VEL_MODEL_PROB': 0.5, 
                            'STOP_PROB': 0.05, 
                            'DISCOUNT_AVG_PROB': 1.0, 
                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                            'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
                            'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
                        }


                        our_method_params_z2.append(one_param_set)
                

In [157]:
res_z2 = evaluate_on_dataset(our_method_params_z2, 'zara2/test/crowds_zara02.txt', 'zara2_vel', limit=500, evaluate_most_likely=False)

Ours - zara2_vel: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:32<00:00,  2.40it/s]


In [160]:
ours_res_z2 = res_z2[0]
trajectron_res_z2 = res_z2[1]

ours_bo20_fde = ours_res_z2['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params_z2[index_min])

0
Lowest FDE:  0.4889130892807272
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 200, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.1, 'VELOCITY_CHANGE_NOISE': 0.05, 'ANGLE_CHANGE_PROB': 0.25, 'ANGLE_CHANGE_NOISE': 2, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


## Univ

In [124]:
our_method_params_univ = []

for angle_change_prob in [0.2]:
    for angle_change_noise in [2.5]: #more or less done (2.5 was the highest and the best)
        for velocity_change_prob in [0.2]: #done
            for velocity_change_noise in [0.1]: #more or less done (0.1 was the smallest and the best)
                for discount_avg_prob in [1.0]: #done
                    one_param_set = {
                        'NOISE': 0.05, 
                        'NO_OF_TRAJECTORIES': 1000, 
                        'CONST_VEL_MODEL_PROB': 0.5, 
                        'STOP_PROB': 0.05, 
                        'DISCOUNT_AVG_PROB': discount_avg_prob, 
                        'DISCOUNT_LOWER_BOUND': 0.2, 
                        'VELOCITY_CHANGE_PROB': velocity_change_prob,
                        'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                        'ANGLE_CHANGE_PROB': angle_change_prob, 
                        'ANGLE_CHANGE_NOISE': angle_change_noise, 
                        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
                        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
                    }


                    our_method_params_univ.append(one_param_set)
                

In [125]:
res_univ = evaluate_on_dataset(our_method_params_univ, 'univ/test/students001.txt', 'univ_vel', limit=2000, evaluate_most_likely=False)


Ours - univ_vel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [02:01<00:00,  3.68s/it]


In [126]:
ours_res_univ = res_univ[0]
trajectron_res_univ = res_univ[1]

ours_bo20_fde = ours_res_univ['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params_univ[index_min])

38
Lowest FDE:  0.4397925446945386
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 1000, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.2, 'VELOCITY_CHANGE_NOISE': 0.1, 'ANGLE_CHANGE_PROB': 0.2, 'ANGLE_CHANGE_NOISE': 2.5, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


## Hotel

In [118]:
our_method_params_hotel = []

for angle_change_prob in [0.2]: #done
    for angle_change_noise in [2]:
        for velocity_change_prob in [0.2]: #more or less done (0.2 was the lowest and the best)
            for velocity_change_noise in [0.1]: #more or less done (0.1 was the lowest and the best)
                for discount_avg_prob in [0.5]: #0.5 was the lowest and the best, should run more
                    one_param_set = {
                        'NOISE': 0.05, 
                        'NO_OF_TRAJECTORIES': 1000, 
                        'CONST_VEL_MODEL_PROB': 0.5, 
                        'STOP_PROB': 0.05, 
                        'DISCOUNT_AVG_PROB': discount_avg_prob, 
                        'DISCOUNT_LOWER_BOUND': 0.2, 
                        'VELOCITY_CHANGE_PROB': velocity_change_prob,
                        'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                        'ANGLE_CHANGE_PROB': angle_change_prob, 
                        'ANGLE_CHANGE_NOISE': angle_change_noise, 
                        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
                        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
                    }


                    our_method_params_hotel.append(one_param_set)
                

In [119]:
res_hotel = evaluate_on_dataset(our_method_params_hotel, 'hotel/test/biwi_hotel.txt', 'hotel_vel', limit=500, evaluate_most_likely=False)


Ours - hotel_vel: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 232/232 [00:41<00:00,  5.61it/s]


In [120]:
ours_res_hotel = res_hotel[0]
trajectron_res_hotel = res_hotel[1]

ours_bo20_fde = ours_res_hotel['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params_hotel[index_min])

18
Lowest FDE:  0.14298221502075015
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 1000, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 0.5, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.2, 'VELOCITY_CHANGE_NOISE': 0.1, 'ANGLE_CHANGE_PROB': 0.2, 'ANGLE_CHANGE_NOISE': 2, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


## ETH

In [154]:
our_method_params_eth = []

for angle_change_prob in [0.25]: #0.25 was the highest and the best
    for angle_change_noise in [2]:
        for velocity_change_prob in [0.2]: #done
            for velocity_change_noise in [0.2]: #done
                for discount_avg_prob in [1.0]: #done
                    for no_of_trajectories in [200, 1000, 2000]:
                        one_param_set = {
                            'NOISE': 0.05, 
                            'NO_OF_TRAJECTORIES': no_of_trajectories, 
                            'CONST_VEL_MODEL_PROB': 0.5, 
                            'STOP_PROB': 0.05, 
                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                            'DISCOUNT_LOWER_BOUND': 0.2, 
                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                            'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
                            'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
                        }


                        our_method_params_eth.append(one_param_set)
                

In [155]:
res_eth = evaluate_on_dataset(our_method_params_eth, 'eth/test/biwi_eth.txt', 'eth_vel', limit=500, evaluate_most_likely=False)



Ours - eth_vel: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 284/284 [01:24<00:00,  3.37it/s]


In [158]:
ours_res_eth = res_eth[0]
trajectron_res_eth = res_eth[1]

ours_bo20_fde = ours_res_eth['BEST_OF_20']['FDE']
index_min = np.argmin(ours_bo20_fde)
print(index_min)
print("Lowest FDE: ", ours_bo20_fde[index_min])
print("Param set: ", our_method_params_eth[index_min])

1
Lowest FDE:  0.48526251574931417
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 1000, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.05, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.2, 'VELOCITY_CHANGE_NOISE': 0.2, 'ANGLE_CHANGE_PROB': 0.25, 'ANGLE_CHANGE_NOISE': 2, 'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]}


In [159]:
print(ours_bo20_fde)

[0.5307309071331852, 0.48526251574931417, 0.5391984758562167]
